This is an example file of us using Talos for hyperparameter optimisation on the diabetes data prediction question

We start by loading the data from the github platform using the loadtxt

In [1]:
from numpy import loadtxt #allows to load data from a text file. Documentation here: https://numpy.org/doc/stable/reference/generated/numpy.loadtxt.html

dataset = loadtxt('https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv', delimiter=',') #delimiter is what separates the diferent values
print(dataset)
print(dataset.shape)

[[  6.    148.     72.    ...   0.627  50.      1.   ]
 [  1.     85.     66.    ...   0.351  31.      0.   ]
 [  8.    183.     64.    ...   0.672  32.      1.   ]
 ...
 [  5.    121.     72.    ...   0.245  30.      0.   ]
 [  1.    126.     60.    ...   0.349  47.      1.   ]
 [  1.     93.     70.    ...   0.315  23.      0.   ]]
(768, 9)


Then we split the data into the x and y variables. x is all the 789 rows but only from column 0 to 7 (first 8 columns). y is the class to which the data belongs - 0 or 1 to mean diabetes or not. So, all the rows (:) but 9th column only (indexed at 8 though) only

In [2]:
x = dataset[:,0:8]
y = dataset[:,8]
print(f'Output')
print(f'------')
print(f'x shape is:', x.shape, 'while x type is:', type(x))
print(f'y shape is:', y.shape, 'while y type is:', type(y))

Output
------
x shape is: (768, 8) while x type is: <class 'numpy.ndarray'>
y shape is: (768,) while y type is: <class 'numpy.ndarray'>


Then we build the classification model. Remember its a deep learning model

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

def diabetes():
    model = Sequential()
    model.add(Dense(10, input_dim = 8, activation = 'relu', kernel_initializer = 'normal'))
    model.add(Dropout(0.2))
    model.add(Dense(8, activation = 'relu'))
    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

    return model

Now we use talos to select set some of the parameters that we want to test. Note that we set them up as a dictionary

In [ ]:
from tensorflow.keras.activations import relu, elu #since these are only available in the library , we have to import the library itself for the model to interpret the dictionary accordingly
#The model can interpret the functions on its own if its fed directly, but now we are not feeding it directly from its own internal dictionary, we are feeding it from the dictionary that we created hence need to import it to our dictionary first
p = {
    'lr':[0.5,5,10],
    'hidden_layers': [0,1,2],
    'epochs':[20,30,40],
    'drop_out_1':[0,0.3,0.8],
    'drop_out_2':[0,0.2,0.4],
    'first_neuron': [12, 24, 36],
    'activation_function_one': ['relu', 'elu'],
    'activation_function_two': ['relu', 'elu'],
    'batch_size':[10,20]
}

At the same time, the library (talos) allows for us to generate multiple parameters automatically then we can clip them when using the .scan

In [ ]:
import talos
p_auto = talos.autom8.AutoParams().params#this generates a dictionary with several elements that we can then modify using the standard dictionary models or clip at the .scan talos method. Note that rerunning this cell generates a new list of values hence the need to probably have this set up somewhere different
p_auto


I will remove certain elements and update certain other ones

In [ ]:
#I want to delete the shapes and network so I define these keys first
keys = ['shapes', 'network']

#Then initialise a for loop to remove them
for key in keys:
    p_auto.pop(key, None)
p_auto

Now we are updating the model function to include these new parameters. Note that we could have ignored the initial building. We only included it to make the process familiar

In [ ]:
def diabetes(X_train, y_train, x_val, y_val, params): #note that at the moment, we don't have x_val and y_val. However, it seems like the model building automatically ignores the validation process if it does not find the validation data itself.
    model = Sequential()
    model.add(Dense(params['first_neuron'], input_dim = 8, activation = 'relu'))
    model.add(Dropout(params['drop_out_1']))
    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

    #since we want to capture the model output, we need to assign it to a variable that the function will return. A very interesting approach when you have the model built inside a function
    out = model.fit(x = x, y = y, validation_data = [x_val, y_val], epochs = params['epochs'], batch_size = params['batch_size'], verbose = 0)

    return out, model

Now the talos experimental run

In [ ]:
import talos #this is the .scan method that I wrote about in the paper.
#the first two arguments make sense. params is the dictionary with the parameters
#model is the name of the model that you just built. In our case, we encased our model into a function hence the name of the function. Otherwise, we would have model being model (since that is the name of our model)
#Experiment_name is used to create the experiment's log folder (where the data is stored)
t = talos.Scan(x = x, y = y, params = p, model = diabetes, experiment_name = 'diabetes')

  0%|          | 3/8748 [01:07<54:54:55, 22.61s/it]






KeyboardInterrupt: 

Now obtaining the results and saving to a dataframe

In [ ]:
#First, we can get the entire dictionary
#These are basically the model iteration and the different model hyperparameters used
exp_dict = t.saved_models
exp_dict

Then we can obtain the useful information that we need and save it to the dataframe

In [ ]:
performance = t.data
performance.head()

Quick overview? Just obtain the model's best performance depending on the score that matters most to you. Though this is a very unidimensional look at the model performance that I would really recommend

In [ ]:
#Here, we are looking at validation accuracy as our calibrating performance indicator. You want to get the highest validation accuracy hence the .max method that is available in the pandas dataframe. In our case, the highest validation accuracy was 78.78 percent with training time of 25 seconds.
performance[performance['val_accuracy'] == performance['val_accuracy'].max()]

THEN WE CAN NOW MOVE TO ANALYSING HOW THE MODEL PERFORMED UNDER DIFFERENT HYPERPARAMETERS. THIS IS THE MORE INFORMED AND MORE DETAILED LOOK INTO THE EXPERIMENT. NOTE THAT WE SAVED OUR DATA TO A DATAFRAME. THIS IS WHAT WE WILL BE USING FROM HERE ONWARDS

In [ ]:
#You could opt to do your analysis with the dataframe using pandas.This would, of course, give you more flexibility especially if you are really great with pandas module. However, talos has various inbuilt functions that you could use for analysis as long as you save the experiment's output into a .Analyze datatype using the .Reporting method with the argument parsed being the object you created for the experiment (in my case, this was t). 
r = talos.Reporting(t)
print(r) #.Analyze datatype. But if you print it, its just an object from which you can obtain different bits like dataframe - .data. Read up more on how to save data into/ retrieve data out of objects in python
r_dataframe = r.data #to extract to a dataframe (method two)
r.high('val_accuracy') #returns highest value in that measurement category. Same as saying .max with the dataframe

Let's try and plot the line graph for val_accuracy

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Times New Roman'

r.plot_line('val_accuracy')
#A bit of detour but when run, error: findfont: Font family 'Geneva' not found pops up. I have looked up several ways to update this to no avail. But generally, it is a line plot that plots val_accuracy against the index

Since we spent too much time on the previous one, we are now moving to plot a correlation graph. Hopefully the font works now because we need it to make sense of the different parameters

WE HAVE TRIED USING THE LIBRARY FOR ANALYSIS BUT IT SEEMS LIKE THERE IS A PROBLEM WITH IT. AS SUCH, WE WILL USE THE DATAFRAME TO PERFORM THE DIFFERENT ANALYSES THAT WOULD, OTHERWISE, HAVE BEEN DONE BY THE METHODS IN THE LIBRARY ITSELF

In [ ]:
#First of all, the r_dataframe contains columns such as start and end which we do not need hence delete. But, we first create a copy of it. We create a deep copy (True) to ensure changes in this dataframe do not affect the original one in case we need it for other purposes
r_dataframe_copy = r_dataframe.copy(deep = True)
r_dataframe_copy.head(10)
#Then we delete the columns that don't matter (non-number columns) using column numbers with first non-index column being column 0
r_dataframe_copy.drop(r_dataframe_copy.columns[[0,1, 9, 10]], axis = 1, inplace = True)

#Then we generate the correlation matrix
corr_matrix = r_dataframe_copy.corr()

#Then import seaborn (for the heatmap) and matplotlib (for displaying the labels)
import seaborn as sn
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Times New Roman'

#Then perform the plot
sn.heatmap(corr_matrix, annot = True) #include annotation
plt.show()
